### Import stuff

In [ ]:
# Importing the packages that are used. Also importing the actual data processing funcitons that have been defined in fl.py
# For more info about how the data is being processed in this script, look in the fl.py document
import fl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

Hello This is how Pia would edit


In [ ]:
# fast reload of external module
import importlib
importlib.reload(fl)

### Pia: Set data path
This is the path to where the data is being stored eg external drive. This path, including subfolders, will be searched for .nd2 files

In [ ]:
data_path = '/Users/jni/Dropbox/share-files/'

In [ ]:
# Creating a dataframe called df_files that contains info about the data. Printing the start and end of the data frame showing 
# the info that it contains including the file name, pixel size, size of the images, number of channels, number of frames (timeponts)
# Channels, frame rate, and position and size of the ROI
df_files= fl.nd2_info_to_df(data_path)
df_files

# Identfying platelet objects in nd2-files

### Create config for processing

In [ ]:
# Here are the settings for the object identification: It uses channel2 (X649) for object identifiction. 
#Settings for DoG and threshold
conf = dict(
            process_type = 'multi_process', # single_thread, multi_process, multi_thread
            multi_workers = 7, # dont use too many
            object_channel = 2,
            intensity_channels = [0, 1],
            dog_sigma1 = 1.7,
            dog_sigma2 = 2.0,
            threshold = 0.15,
            peak_min_dist = 3,
            z_dist = 2,
            center_roi = True,
            rotate = True,
            rotate_angle = 45,
            )

### Process loop 
Saves object dataframe and config file at location of the nd2-file. This is the most computationally heavy part of the analysis where objets are identified. For our standard tracking injuries it takes about 15 min per injury with the current settings.

In [ ]:
#Pia: List the number of files list (range(XX)) in the data set that will be processed. If Whole data set use list(range(xx)) 
# If example injury use [X] (put hashtag on the one that you are not using).
# Creating a new data frame for each injury called df_obj and these are saved as pickle files: df_filename

fileIds_to_process = [0] 
#fileIds_to_process = list(range(66)) 
ivmObjects = fl.IvmObjects(conf)

# loop for processing multiple nd2-files
now_start = fl.get_datetime()
time = Path(fl.get_datetime())
time.mkdir(exist_ok=True)

for fileId in fileIds_to_process:
    # process file
    file, frames = df_files.iloc[fileId][['file', 't']]
    ivmObjects.add_nd2info(df_files.iloc[fileId]) #add nd2-file info to conf
    
    df_obj = ivmObjects.process_file(file, range(frames))#frames))
    
    #--------------------------------------------------------
    #Niklas changed this section to change name of file and directory
    file_path=Path(file)
    now = fl.get_datetime()
    df_filename = f'./{now_start}/{file_path.stem}.{now}.df.pkl'
    conf_filename = f'./{now_start}/{file_path.stem}.{now}.conf.yml'
    
    #--------------------------------------------------------
    
    # save result
    df_obj.to_pickle(df_filename)
    fl.save_yaml(conf, conf_filename)
    

In [ ]:
df_obj.head()

In [ ]:
df_obj.shape

# Inspect image processing pipeline - here we can look at the outcome of the image processing for a selected injury

### Create config for processing

In [ ]:
conf = dict(
            process_type = 'single_thread', # USE ONLY 'single_thread' for inspection
            multi_workers = 7, # dont use too many
            object_channel = 2,
            intensity_channels = [0, 1],
            dog_sigma1 = 1.7,
            dog_sigma2 = 2.0,
            threshold = 0.15,
            peak_min_dist = 3,
            z_dist = 2,
            center_roi = True,
            rotate = True,
            rotate_angle = 45,
            )

In [ ]:
df_files= fl.nd2_info_to_df(data_path)
df_files

In [ ]:
#df_files['file']=Path(df_files.file).stem

### Process selected volume

In [ ]:
# Pia, select injury (file id) and time point (frame) that you want to visualise for effect of image processing
fileId = 0
frame = 70

ivmObjects = fl.IvmObjects(conf)
file, frames = df_files.iloc[fileId][['file', 't']]
ivmObjects.add_nd2info(df_files.iloc[fileId]) #add nd2-file info to conf
df_obj_insp = ivmObjects.process_file(file, [frame])



    

In [ ]:
df_obj_insp.head()

## Inspect image processing steps - figure showing result of each image processing step.

In [ ]:
# list steps
insp_steps = ivmObjects.inspect_steps
step_names = [(stp, insp_steps[stp]['name']) for stp in insp_steps]
print(step_names)

In [ ]:
z_levels = [0, 5, 10, 15, 20]
figsize = (15,15)


cols = len(insp_steps)
rows = len(z_levels)

fig, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)
for r, z in enumerate(z_levels):
    for c, stp in enumerate(insp_steps):
        axs[r, c].imshow(insp_steps[stp]['data'][...,z],
                         vmin=insp_steps[stp]['data'].min(),
                         vmax=insp_steps[stp]['data'].max(),
                        cmap='nipy_spectral') 
        
        axs[r, c].get_xaxis().set_ticks([])
        axs[r, c].get_yaxis().set_ticks([])
        
        if r == 0:
            axs[r, c].set_title(insp_steps[stp]['name'])
        if c == 0:
            axs[r, c].set_ylabel(z)
plt.tight_layout()

In [ ]:
import napari

In [ ]:
%gui qt

In [ ]:
z_level = 5
fl.imshow((insp_steps[0]['data'][...,z_level])

## Inspect object positions - plot object positions at every 20th frame (approximately every min if frame rate 0.32). Chose which property to show in color (eg Calcium mean intensity)

In [ ]:

plt.rcParams['image.cmap'] = 'coolwarm'
#plt.rcParams['image.cmap'] = 'jet_r'

#Gor lista av dimensioner 
lims=['xs', 'ys', 'zs']
limsv={}

#Anger gransvarden for de olika dimensionerna genom att kolla minsta och storsta varde och plussa pa 40
border=1.5
for l in lims:
    limsv[l]=df_obj[l].min()-20, df_obj[l].max()+20
    
col='path'
row='c'
hue='c'
x='frame'
y='z'
unit='c'

#Tar ut var 20:e frame fran frame 2 och framat
frames=pd.unique(df_obj.frame)[::20]+1
ncols=3
nrows=len(frames)

#Gor en lagom stor figur
plt.figure(figsize=(ncols*4,nrows*3))

#Valjer dimensioner att plotta 
cols=[('xs', 'ys'), ('xs', 'zs'), ('ys', 'zs')]

### name väljer variabel att färgsätta plottarna med
#name='cld'#name='stab'#name='c'#name='depth' #colorv=[1,2,4,8] #name='c2_max'
name='int_mean'

#vmin=0 #vmax=30#vmin=0
#vmax=10 #vmax=400

for r, f in enumerate(frames):
    sel_f=df_obj[df_obj.frame==f]
    
    for c, xy in enumerate(cols):
        ax=plt.subplot2grid((nrows, ncols), (r, c))
        #plt.setp()
        ax.scatter(sel_f[xy[0]], sel_f[xy[1]], alpha=0.5, c=sel_f[name], linewidth=0.1 )#, vmin=vmin, vmax=vmax,)
        ax.set_title('Time (sec): '+ str(np.round(sel_f.time.mean())),fontsize=12)
        ax.set_ylim(limsv[xy[1]])
        ax.set_xlim(limsv[xy[0]])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.tick_params(labelsize=12)
        #ax.ticklabel_format()
        #ax.set_axis_bgcolor('black')
plt.tight_layout()


#fname='e__' + name + '.png'

#plt.savefig(fname, bbox_inches='tight', dpi=300)